In [6]:
import os

# Ensure the directory exists
os.makedirs('Model', exist_ok=True)
os.makedirs('Excel', exist_ok=True)
os.makedirs('Gambar', exist_ok=True)
os.makedirs('Gambar/Heatmap', exist_ok=True)
os.makedirs('Gambar/Best Worst', exist_ok=True)

In [7]:
import numpy as np

# Load Datasets
test_indices = np.load('Array/test_indices.npy')
train_indices = np.load('Array/train_indices.npy')

# Load the data
data_x = np.load('input_x.npy')
data_y = np.load('input_y.npy')

# Split the data into training and testing sets
x_train_true = data_x[train_indices]
y_train_true = data_y[train_indices]
x_test_true = data_x[test_indices]
y_test_true = data_y[test_indices]

import torch
import torch.optim as optim
import torch.nn as nn

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Convert data to tensor if needed and move to the correct device
Tensor_X = torch.tensor(x_test_true, dtype=torch.float32).to(device)

# Define your model architecture
class ANNModel(nn.Module):
    def __init__(self, input_dim, hidden_dims, output_dim, dropout_rate=0.3):
        super(ANNModel, self).__init__()
        layers = []
        prev_dim = input_dim
        for h_dim in hidden_dims:
            layers.append(nn.Linear(prev_dim, h_dim))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout_rate))  # Dropout to prevent overfitting
            prev_dim = h_dim
        layers.append(nn.Linear(prev_dim, output_dim))  # Output layer (no activation for regression)
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)

# Load the best hyperparameters
best_hyperparameters = torch.load('Model/Best_Hyperparameters.pth')

# Extract each parameter
best_hidden_layers = best_hyperparameters['hidden_layers']
best_hidden_units = [best_hyperparameters[f'n_units_l{i}'] for i in range(best_hidden_layers)]
best_dropout_rate = best_hyperparameters['dropout_rate']
best_lr = best_hyperparameters['lr']
best_optimizer_name = best_hyperparameters['optimizer']

# Re-initialize the model with the loaded best hyperparameters
model = ANNModel(
    input_dim=8,
    hidden_dims=best_hidden_units,
    output_dim=462,
    dropout_rate=best_dropout_rate
).to(device)  # Ensure 'device' is defined (e.g., device = torch.device("cuda" if torch.cuda.is_available() else "cpu"))

# Set up the optimizer using the loaded hyperparameter
best_optimizer = getattr(optim, best_optimizer_name)(model.parameters(), lr=best_lr)

# Define the loss criterion
criterion = nn.MSELoss()

# Load the best model weights
model.load_state_dict(torch.load('Model/ANN_Model.pt'))
model.eval()  # Set the model to evaluation mode

# Predict with the model
with torch.no_grad():  # No gradients needed for inference
    predictions = model(Tensor_X)

# Convert predictions to a numpy array or DataFrame as needed
y_pred = predictions.cpu().numpy()

# Create Pytorch Datasets

Tensor_X = torch.tensor(x_test_true, dtype=torch.float32)
Tensor_X.to(device)

C:\Users\Athal\AppData\Local\Temp\ipykernel_30424\3300693080.py:45: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  best_hyperparameters = torch.load('Model/Best_Hyperparamete

tensor([[0.0000, 0.0000, 0.3333, 0.7785, 0.0469, 0.0769, 0.4218, 1.0000],
        [0.0000, 0.0000, 0.3333, 0.3627, 0.3750, 0.2115, 0.2007, 0.8000],
        [0.0000, 0.0000, 0.3333, 0.3671, 0.7344, 0.1538, 0.3946, 0.3667],
        [0.0000, 0.0000, 0.3333, 0.2726, 0.0000, 0.0385, 0.6803, 0.9333],
        [0.0000, 0.0000, 0.6667, 0.2189, 0.7344, 1.0000, 0.9796, 0.0000],
        [0.0000, 0.0000, 0.6667, 0.2726, 0.0000, 0.0385, 0.6803, 0.9333],
        [0.0000, 0.0000, 0.6667, 0.2233, 0.2188, 0.1731, 0.7415, 0.6667],
        [0.0000, 0.0000, 1.0000, 1.0000, 0.7500, 0.0577, 0.3605, 0.3000],
        [0.0000, 0.0000, 1.0000, 0.4171, 1.0000, 0.9038, 0.0000, 0.2000],
        [0.0000, 0.0000, 1.0000, 0.5820, 0.6094, 0.7115, 1.0000, 0.1333],
        [0.0000, 0.0000, 1.0000, 0.3671, 0.7344, 0.1538, 0.3946, 0.3667],
        [0.0000, 1.0000, 0.0000, 1.0000, 0.7500, 0.0577, 0.3605, 0.3000],
        [0.0000, 1.0000, 0.0000, 0.2189, 0.7344, 1.0000, 0.9796, 0.0000],
        [0.0000, 1.0000, 0.3333, 0.417

In [8]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

# Calculate metrics definition
def calculate_metrics(true, pred):
    r2 = r2_score(true, pred, multioutput='uniform_average')
    mse = mean_squared_error(true, pred)
    mae = mean_absolute_error(true, pred)
    return r2, mse, mae

# Calculate score
r2_train, mse_train, mae_train = calculate_metrics(y_test_true, y_pred)


# Output the results
print("\n===== Model Performance =====")
print(f"  R2 Score: {r2_train:.4f}")
print(f"  MSE: {mse_train:.4f}")
print(f"  MAE: {mae_train:.4f}")



===== Model Performance =====
  R2 Score: 0.8623
  MSE: 0.0056
  MAE: 0.0415
